## OSMOSIS DEX DATA

Data source: https://api-osmosis.imperator.co/swagger/#/

06/09/2022

We need: 
- 6-month average daily volume
- 6-month average TVL
- Largest pool concentration
- Liquidity pools with > 1mn TVL
- 6 month average FDV:TVL
- Monthly active users
- Average trade execution time
- Implied Slippage
- MEV

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt 
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings(action="ignore")

### 6-month average daily volume

In [2]:
def get_6month_average_volume():
    url = 'https://api-osmosis.imperator.co/volume/v2/historical/chart'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df['time'] = pd.to_datetime(df['time'])
        return df['value'].loc[df['time'] >= (dt.datetime.today() - relativedelta(months = 6))].sum()/len(pd.date_range(start = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0) - relativedelta(months = 6)), end = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0)), freq= 'D'))
    else:
        raise KeyError('Endpoint could be fucked!')

In [3]:
get_6month_average_volume()

42737571.145576656

### 6-month average TVL

In [4]:
def get_6month_average_TVL():
    url = 'https://api-osmosis.imperator.co/liquidity/v2/historical/chart'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df['time'] = pd.to_datetime(df['time'])
        return df['value'].loc[df['time'] >= (dt.datetime.today() - relativedelta(months = 6))].sum()/len(pd.date_range(start = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0) - relativedelta(months = 6)), end = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0)), freq= 'D'))
    else:
        raise KeyError('Endpoint could be fucked!')

In [5]:
avgtvl = get_6month_average_TVL()
avgtvl

614450205.8819946

### Largest pool concentration 

In [6]:
def largest_pool_concentration_TVL():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')
        ddf['proportionTVL'] = ddf['liquidity'] / ddf['liquidity'].sum()
        
        largest_prop = ddf['proportionTVL'].max()
        top10_prop = ddf['proportionTVL'].nlargest(10).sum()

        return [largest_prop, top10_prop]
    else:
        raise KeyError('Endpoint could be fucked!')
    

In [7]:
largest_pool_concentration_TVL()

[0.39311949880955976, 0.8427253141222584]

In [8]:
def largest_pool_concentration_7d_volume():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')
        ddf['proportionVol'] = ddf['volume_7d'] / ddf['volume_7d'].sum()
        
        largest_prop = ddf['proportionVol'].max()
        top10_prop = ddf['proportionVol'].nlargest(10).sum()

        return [largest_prop, top10_prop]
    else:
        raise KeyError('Endpoint could be fucked!')

In [9]:
largest_pool_concentration_7d_volume()

[0.3168231901744671, 0.8819459808131513]

### Liquidity pools with > 1mil USD  

In [10]:
def no_pools_gte_1mUSD():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')

        return len(ddf.loc[ddf['liquidity'] >= 1000000])
    else:
        raise KeyError('Endpoint could be fucked!')

In [11]:
no_pools_gte_1mUSD()

19

### Fee Spread

In [12]:
def liquid_pair_fee_spread():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
    
        mini = df['fees'].apply(lambda x: float(x.replace('%', ''))).min()
        maxi = df['fees'].apply(lambda x: float(x.replace('%', ''))).max()

        return [mini, maxi]
    else:
        raise KeyError('Endpoint could be fucked!')

In [13]:
liquid_pair_fee_spread()

[0.04, 0.534]

### 6 month average FDV/TVL 

In [ ]:
#FDV is the blocker

In [ ]:
url = 'https://api.cosmostation.io/v1/status'
response = requests.get(url)

In [ ]:
response.ok

In [ ]:
data = response.json()

In [ ]:
data['total_supply_tokens']

In [ ]:
total_supply = pd.DataFrame(data['total_supply_tokens']['supply'])

In [ ]:
url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
response = requests.get(url, headers= {'accept': 'application/json'})
if response.ok:
    data = response.json()

    df = pd.DataFrame()
    for key in data.keys():
        tdf = pd.DataFrame(data[key])
        tdf['pool_id'] = key
        df = df.append(tdf)

    ddf = df.drop_duplicates(subset='pool_id')
    ids = list(ddf['pool_id'].loc[ddf['liquidity'].isin(ddf['liquidity'].nlargest(10))])

In [ ]:
tdf = df.loc[df['pool_id'] == ids[1]]

In [ ]:
tdf

In [ ]:
total_supply.loc[total_supply['denom'] == tdf['denom'].iloc[0]]

### Montly active users

### Average Trade Exection Time

Block time: https://www.mintscan.io/cosmos/validators

6.86s 

### Implied Slippage 

In [14]:
def balancer_slippage(tdf, dollar_value):
    ep = tdf['amount'].iloc[-1]/tdf['amount'].iloc[0]
    token0amountin = dollar_value/(tdf['price'].iloc[0])
    token1out = token0amountin*(1-(tdf['amount'].iloc[0]/(tdf['amount'].iloc[0]+tdf['amount'].iloc[-1])))
    nep = (tdf['amount'].iloc[-1] - token1out)/(tdf['amount'].iloc[0]+ token0amountin)
    slipage = abs(ep/nep -1)
    return slipage

In [15]:
def get_slippage():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)

        ddf = df.drop_duplicates(subset='pool_id')
        ids = list(ddf['pool_id'].loc[ddf['liquidity'].isin(ddf['liquidity'].nlargest(10))])

        slip1000 = []
        slip10000 = []
        slip100000 = []
        for i in range(len(ids)):
            tdf = df.loc[df['pool_id'] == ids[i]]
            slip1000.append(balancer_slippage(tdf, 1000))
            slip10000.append(balancer_slippage(tdf, 10000))
            slip100000.append(balancer_slippage(tdf, 100000))
            
        return  slip1000, slip10000, slip100000

    else:
        raise KeyError('Endpoint could be fucked!')


In [16]:
s1, s2, s3 = get_slippage()

In [17]:
np.mean(s1)

0.0004057317422300866

In [18]:
np.mean(s2)

0.00406675104618055

In [19]:
np.mean(s3)

0.041647486378403986

### MEV